# 87651260 Applications of Geographic Information System

# Geolocation

## Attawut Nardkulpat, Reseach Officer, Burapha University
### attawut@buu.ac.th
### 14/08/2020

In [ ]:
!pip install geopandas
!pip install GeoText
!pip install geopy

In [ ]:
import matplotlib as plt
%matplotlib inline
import pandas as pd
import matplotlib.patches as mpatches
import plotly.express as px
from geotext import GeoText
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import requests
from bs4 import BeautifulSoup
import os
import re
from functools import reduce

In [ ]:
df2 = pd.read_csv('2020-03-04 Coronavirus Tweets.CSV',encoding='utf-8')
df2_text = df2[["text"]]
df2_text.to_string()
df2_text1 = df2_text.reset_index(drop=True)
df2_text1['geotext'] = df2_text1["text"].apply(lambda text: GeoText(text))

# Extract Location

In [ ]:
a = []
for i in range(0,len(df2_text1)):a  += (df2_text1['geotext'][i].cities)
city_mentions = pd.DataFrame(a, columns=['city'])

In [ ]:
city_mentions

In [ ]:
rank = city_mentions['city'].value_counts(ascending = False)
rank

In [ ]:
cities_to_remove = ['Corona', 'March', 'Of','Most','Obama','Un','Una','Bo','Best','Ho','Nice','University','Man','Police','Como']

In [ ]:
city_mentions = city_mentions[~city_mentions['city'].isin(cities_to_remove)]
city_mentions

In [ ]:
rank = city_mentions['city'].value_counts()
rank

In [ ]:
rank = pd.DataFrame(rank)
rank

In [ ]:
rank.to_csv('test_rank.csv')

# Extract Geolocation

In [ ]:
df = pd.read_csv("test_rank.csv",encoding='utf-8')
df

In [ ]:
df.columns = ['city','count']

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [ ]:
#use geopy's geolocator to find information on all the cities in the list
chrome_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36"

geolocator = Nominatim(timeout=10,user_agent=chrome_user_agent)

lat_lon = []
for i in df.city: 
    try:
        location = geolocator.geocode(i)
        if location:
            lat_lon.append(location)
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input %s with message %s"%
             (i, e))

In [ ]:
#create a df for raw output from the geolocator, maybe put cleaning directly after this; or append to original df
city_data = pd.DataFrame(lat_lon, columns=['raw_data','raw_data2'])

#re-order the cols
city_data = city_data[['raw_data2', 'raw_data']]

#get the city names from raw_data
city_data['city'] = city_data['raw_data'].str.split(',').str[0]

In [ ]:
#change the coordinates to a string
city_data['raw_data2'] = city_data['raw_data2'].astype(str)

#split the coordinates using the comma as the delimiter
city_data[['lat','lon']] = city_data.raw_data2.str.split(",",expand=True,)

#remove the parentheses
city_data['lat'] = city_data['lat'].map(lambda x:x.lstrip('()'))
city_data['lon'] = city_data['lon'].map(lambda x:x.rstrip('()'))

#convert the lat and lon columns back to floats
city_data = city_data.astype({'lat': 'float64', 'lon': 'float64'})

#drop the raw data columns
city_data.drop(['raw_data2', 'raw_data'], axis = 1, inplace=True)

In [ ]:
city_data

In [ ]:
#merge the two dataframes
merged = pd.merge(df, city_data, left_index=True, right_index=True)

In [ ]:
merged

In [ ]:
merged.to_csv("geolocation_test.csv")